In [1]:
#!pip install meetup.api

  Using cached https://files.pythonhosted.org/packages/b7/44/a545b860f19cac088cb7f3e39beae8334b63e025729348ddeffb3342122a/meetup_api-0.1.1-py2.py3-none-any.whl


twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [1]:
import meetup.api
import json 
import requests 
import time 
import codecs
import sys
import io
import re
import pandas as pd
import sqlalchemy as sa

In [3]:
client = meetup.api.Client('1a60703a102a6252424875415c1b21')

In [4]:
group_info = client.GetGroup({'urlname': 'DC-Tech-Meetup'})

29/30 (10 seconds remaining)


In [5]:
group_info.id

1770462

In [6]:
df = requests.get('https://api.meetup.com/2/members?group_id=1770462&key=1a60703a102a6252424875415c1b21')

In [7]:
df.status_code

200

In [8]:
df.headers['content-type']

'application/json;charset=utf-8'

In [9]:
df.encoding

'utf-8'

In [10]:
df.json()

{'results': [{'country': 'us',
   'city': 'Washington',
   'topics': [{'urlkey': 'diningout', 'name': 'Dining Out', 'id': 713},
    {'urlkey': 'softwaredev', 'name': 'Software Development', 'id': 3833},
    {'urlkey': 'newtech', 'name': 'New Technology', 'id': 9696},
    {'urlkey': 'web', 'name': 'Web Technology', 'id': 10209},
    {'urlkey': 'social-media', 'name': 'Social Media', 'id': 15401},
    {'urlkey': 'online-marketing', 'name': 'Online Marketing', 'id': 15585},
    {'urlkey': 'foodie', 'name': 'Foodie', 'id': 18528},
    {'urlkey': 'user-experience', 'name': 'User Experience', 'id': 19243},
    {'urlkey': 'collaboration', 'name': 'Collaboration', 'id': 21067},
    {'urlkey': 'startup-businesses',
     'name': 'Startup Businesses',
     'id': 21681},
    {'urlkey': 'professional-women',
     'name': 'Professional Women',
     'id': 26273},
    {'urlkey': 'data-analytics', 'name': 'Data Analytics', 'id': 30928},
    {'urlkey': 'technology-startups',
     'name': 'Technology Sta

In [11]:
type(df.json())

dict

In [17]:
def superFetch(groupname, key):
    print("Start fetching")
    client = meetup.api.Client(key)
    print('request status:',end=' ')
    group = client.GetGroup({'urlname':groupname})
    gid = group.id
    total = group.members
    want = 'id,name,state,city,lon,lat'
    limit = 200
    request = total//limit+1
    offsetValue = 0
    name = []
    uid = []
    state = []
    city = []
    lon = []
    lat = []
    for i in range(request):
        print('progress: ',offsetValue+1,'/',request, sep='')
        print('request status:',end=' ')
        try:
            members = client.GetMembers(group_id=gid, only=want,page=limit,offset=offsetValue)
            offsetValue+=1
            rList= members.results
            for j in rList:
                if len(j.keys())==4:
                    name.append(j['name'])
                    uid.append(j['id'])
                    state.append(j['state'])
                    city.append(j['city'])
                else:
                    uid.append(j['id'])
                    try:
                        name.append(j['name'])
                    except KeyError:
                        name.append('null')
                    try:
                        state.append(j['state'])
                    except KeyError:
                        state.append('null')
                    try:
                        city.append(j['city'])
                    except:
                        city.append('null')
                    try:
                        lon.append(j['lon'])
                        lat.append(j['lat'])
                    except:
                        lon.append('null')
                        lat.append('null')
        except json.decoder.JSONDecodeError:
            print("JSONDecodeError")
    print("fetching complete")
    df = pd.DataFrame({"name":name, "uid":uid, "state":state, "city":city, "lon":lon, "lat":lat})
    return df


key = '1a60703a102a6252424875415c1b21'
df = superFetch('DC-Tech-Meetup', key)

Start fetching
request status: 29/30 (10 seconds remaining)
progress: 1/125
request status: 28/30 (10 seconds remaining)
progress: 2/125
request status: 27/30 (7 seconds remaining)
progress: 3/125
request status: 26/30 (4 seconds remaining)
progress: 4/125
request status: 25/30 (1 seconds remaining)
progress: 5/125
request status: 29/30 (10 seconds remaining)
progress: 6/125
request status: 28/30 (7 seconds remaining)
progress: 7/125
request status: 27/30 (4 seconds remaining)
progress: 8/125
request status: 26/30 (1 seconds remaining)
progress: 9/125
request status: 29/30 (10 seconds remaining)
progress: 10/125
request status: 28/30 (7 seconds remaining)
progress: 11/125
request status: 27/30 (4 seconds remaining)
progress: 12/125
request status: 26/30 (1 seconds remaining)
progress: 13/125
request status: 29/30 (10 seconds remaining)
progress: 14/125
request status: 29/30 (10 seconds remaining)
progress: 15/125
request status: 28/30 (7 seconds remaining)
progress: 16/125
request stat

In [19]:
def scrapeData(protocol, host, resource):
    '''
    This function takes elements of a URL and returns the raw text from
    the webpage.
    Parameters: protocol, host, resource
    Returns: response text
    '''
    template = "{}://{}{}"
    url = template.format(protocol, host, resource)
    resp = requests.get(url)
    if resp.status_code == 200:
        return resp.text
    print("Error")

txt = scrapeData("https", "en.wikipedia.org", "/wiki/Washington_metropolitan_area")

In [20]:
def extractSectionBetween(txt, start, end):
    '''
    Slices text by substring boundaries rather than index boundaries.
    Parameters: text to slice, start string, end string
    Returns: trimmed string
    '''
    startInd = txt.find(start)
    endInd = txt.find(end)
    return txt[startInd:endInd]

txt = extractSectionBetween(txt, "92.3 years", "The Washington, D.C. area has the largest science")

In [23]:
def recreateTable(txt):
    '''
    Takes text containing a table in raw HTML format and
    places desired data in a pandas data frame.
    Parameters: html text
    Returns: pandas data frame
    '''
    pat = re.compile(r"<td><a .*?>(?P<county>.*?)<\/a>\n<\/td>\n<td .*?>(?P<pop>.*?)\n<\/td>")
    countyid = []
    i = 1
    county = []
    state = []
    pop = []
    for m in pat.finditer(txt):
        county.append(m["county"].split(",")[0].strip())
        state.append(m["county"].split(",")[1].strip())
        pop.append(int(m["pop"].replace(",", "")))
        countyid.append(i)
        i += 1
    return pd.DataFrame({"CountyID":countyid, "County":county, "State":state, "Population":pop})
    
popTable = recreateTable(txt)

popTable["State"] = popTable["State"].replace("D.C.", 1)
popTable["State"] = popTable["State"].replace("Maryland", 2)
popTable["State"] = popTable["State"].replace("Virginia", 3)



print(popTable)
    


    CountyID                  County  State  Population
0          1              Washington      1      681170
1          2          Calvert County      2       91251
2          3          Charles County      2      157705
3          4        Frederick County      2      247591
4          5       Montgomery County      2     1043863
5          6  Prince George's County      2      908049
6          7              Alexandria      3      155810
7          8        Arlington County      3      230050
8          9           Clarke County      3       14374
9         10         Culpeper County      3       50083
10        11          Fairfax County      3     1138652
11        12            Fairfax City      3       24164
12        13            Falls Church      3       14014
13        14         Fauquier County      3       69069
14        15          Fredericksburg      3       28297
15        16          Loudoun County      3      385945
16        17                Manassas      3     

In [3]:
def getCreds(filename,subset,defaults={}):
    '''
    This function helps us connect to a database on hadoop2.  The
    userid password information is stored in a local file encoded
    in json format.  
    Parameters:
    filename is where json encoded userid/password information stored
    subset: dictionary key in json file
    defaults: alternative way to connect to database
    Returns: subdictionary based on subset parameter.
    '''
    try:
        with open(filename,'r') as file:
            D = json.load(file)
            file.close()
            if D[subset]:
                return D[subset]
            else:
                return defaults
    except:
        return defaults

def db_setup(userid,password):
    '''
    This function connects us to a database using our login credentials. 
    Paramters:
    userid - string with our userid
    password - string with our password
    database - string with database name
    Returns:
    SQL engine and SQL connection objects
    '''
    template = 'mysql+mysqlconnector://{}:{}@hadoop2.mathsci.denison.edu/'
    cstring = template.format(userid,password)
    engine = sa.create_engine(cstring)
    connection = engine.connect()
    
    return engine, connection

creds = getCreds('creds.json','mysql',defaults = {'user':'studen_j1', 'password':'studen_j1'})

In [8]:
try:
    connection.close()
    del engine
except:
    pass
engine, connection = db_setup(creds['user'],creds['password'])

In [ ]:
makeDB = "CREATE DATABASE meetup;"
#connection.execute(makeDB)

In [ ]:
#connection.execute("USE meetup;")

makeCountyPopulations = "CREATE TABLE CountyPopulations(\
CountyID INT NOT NULL,\
CountyName VARCHAR(30),\
StateID INT,\
Population INT,\
PRIMARY KEY(CountyID)\
);"

makeStates = "CREATE TABLE States(\
StateID INT NOT NULL,\
State VARCHAR(30),\
PRIMARY KEY(StateID)\
);"

makeUsers = "CREATE TABLE Users(\
MemberID INT NOT NULL,\
Nickname VARCHAR(30),\
StateID INT,\
UserCity VARCHAR(30),\
UserLongitude DECIMAL(5, 2),\
UserLatitude DECIMAL(5, 2),\
PRIMARY KEY(StateID)\
);"

#connection.execute(makeCountyPopulations)
#connection.execute(makeStates)
#connection.execute(makeUsers)


In [16]:
countyRowTemplate = "INSERT INTO CountyPopulations\
(CountyID, CountyName, StateID, Population)\
VALUES({},{},{},{});"

stateRowTemplate = "INSERT INTO States\
(StateID, State)\
VALUES({},{});"

usersRowTemplate = "INSERT INTO Users\
(MemberID, Nickname, StateID, UserCity, UserLongitude, UserLatitude)\
VALUES({},{},{},{},{},{});"

for i in length(popTable):
    countyRow = countyRowTemplate.format(popTable.iloc[i, 0], "'"+popTable.iloc[i, 1]+"'", popTable.iloc[i, 2], popTable.iloc[i, 3])
    
    


In [12]:
try:
    connection.close()
    del engine
    print("Connection closed")
except:
    pass
    print("No connection")

No connection
